# Generate the (recency-weighted) rating matrices
(can be skipped if the files are already available)

In [ ]:
import pandas as pd
import pickle

from models.actr import *
from models.uknn import *

from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix, vstack

import sys
from utils.utils import load_file
sys.path.append('..')
from paths.paths import *
datadir = f"{BASE_FOLDER}/necessary_files/"

In [ ]:
# get users, needed to be ordered by ID, get items

with open(datadir+"users_above_threshold.pkl", "rb") as f:
    users_above_threshold = pickle.load(f)

with open(datadir+'items_above_threshold_and_pad_2.pkl', 'rb') as f:
     items_above_threshold = pickle.load(f)



users_above_threshold.sort()
users = users_above_threshold

# df_users = pd.DataFrame(users)
# df_users.to_csv(datadir+"user_list.csv")

# Generate user-item matrix from train data

In [ ]:
# Store the listening events of every user as an array
# for later conversion to a matrix

listi = []
for test_user in users:
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    col_names = ['user', 'track_id', 'album_id', 'timestamp']
    events = pd.read_csv(filename, names=col_names,
                         quoting=3, sep="\t", header=None, encoding='utf-8'
                         )
    events = events[events["track_id"].isin(items_above_threshold)]

    listi = listi + (events['track_id'].unique().tolist())

listi = list(dict.fromkeys(listi))

In [ ]:
df_dummies = pd.DataFrame(listi, columns=['track_id'])
df_dummies['value'] = [0 for _ in range(len(df_dummies))]

#start with 1 user and create dictionary, listening events will get value 1, non listening 0 via dummies, 
#events_ will be the user rating matrix
for test_user in [2]:
    df_dummies['user'] = [test_user for _ in range(len(df_dummies))]
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    events = load_file(filename)
    events = events[events["track_id"].isin(items_above_threshold)]
    user_interactions = len(events)
    n_test_int = user_interactions // 5
    train, val, test = events[:-2 * n_test_int], events[-2 * n_test_int:-n_test_int], events[-n_test_int:]
    events = train
    events['value'] = [1 for _ in range(len(events))]
    events = events[['track_id', 'value', 'user']]
    events = events.append(df_dummies)
    events_ = events.groupby(['user', 'track_id'])['value'].aggregate('sum').unstack(fill_value=0)
    dictionary_track_id = pd.DataFrame(events_.columns.values.tolist(), columns=['track_id'])
    dictionary_track_id.to_csv('dictionary_track_id_train.csv')

    events_ = events_.to_numpy()
    events_ = events_.reshape(1, -1)
    events_ = coo_matrix(events_)

users.remove(2)
for test_user in users:
    df_dummies['user'] = [test_user for _ in range(len(df_dummies))]
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    events = load_file(filename)
    events = events[events["track_id"].isin(items_above_threshold)]
    user_interactions = len(events)
    n_test_int = user_interactions // 5
    train, val, test = events[:-2 * n_test_int], events[-2 * n_test_int:-n_test_int], events[-n_test_int:]
    events = train
    events['value'] = [1 for _ in range(len(events))]
    events = events[['track_id', 'value', 'user']]
    events = events.append(df_dummies)
    events = events.groupby(['user', 'track_id'])['value'].aggregate('sum').unstack(fill_value=0)
    events = events.to_numpy()
    events = events.reshape(1, -1)
    events = coo_matrix(events)
    events_ = vstack([events_, events])
    events_ = coo_matrix(events_)

In [ ]:
events_ = csr_matrix(events_)
sparse.save_npz(datadir+"user_item_matrix_train.npz", events_)

# Generate ACT-R user-item matrix from train data

In [ ]:

bll = BaseLevelComponent(decay=0.5)
assoc = AssociativeComponent()


def actr_score(bll, assoc, train, bpr='no_bpr'):
    score_bll = bll.score(train)
    score_assoc = assoc.score(train)
    score_list = [score_assoc, score_bll]
    scores = aggregate_scores(score_list, method='scoring')

    return scores


USE_CONTENT = False
CALC_REWARD = False
durations = None


In [ ]:
#Actr-scores matrix of the train set (user-item matrix)

listi = []  #this collects all items of any user and inserts 0 and will be appended to each users history to get the full user-item matrix
for test_user in users:
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    col_names = ['user', 'track_id', 'album_id', 'timestamp']
    events = pd.read_csv(filename, names=col_names,
                         quoting=3, sep="\t", header=None, encoding='utf-8'
                         )
    events = events[events["track_id"].isin(items_above_threshold)]

    listi = listi + (events['track_id'].unique().tolist())

listi = list(dict.fromkeys(listi))

df_dummies = pd.DataFrame(listi, columns=['track_id'])
df_dummies['value'] = [0 for _ in range(len(df_dummies))]
print(df_dummies)

for test_user in [2]:
    df_dummies['user'] = [test_user for _ in range(len(df_dummies))]
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    events = load_file(filename)
    events = events[events["track_id"].isin(items_above_threshold)]
    user_interactions = len(events)
    n_test_int = user_interactions // 5
    train, val, test = events[:-2 * n_test_int], events[-2 * n_test_int:-n_test_int], events[-n_test_int:]
    # events = preprocess(train)
    scores = actr_score(bll, assoc, train)
    events = scores.reset_index()
    events['user'] = [test_user for _ in range(len(events))]
    events['track_id'] = events['index']
    events['value'] = events[0]
    events = events[['track_id', 'value', 'user']]
    events = events.append(df_dummies)
    events_ = events.groupby(['user', 'track_id'])['value'].aggregate('sum').unstack(fill_value=0)
    dictionary_track_id = pd.DataFrame(events_.columns.values.tolist(), columns=['track_id'])
    #dictionary_track_id.to_csv('dictionary_track_id_train.csv')

    events_ = events_.to_numpy()
    events_ = events_.reshape(1, -1)
    events_ = coo_matrix(events_)

users.remove(2)

for test_user in users:
    df_dummies['user'] = [test_user for _ in range(len(df_dummies))]
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    events = load_file(filename)
    events = events[events["track_id"].isin(items_above_threshold)]
    user_interactions = len(events)
    n_test_int = user_interactions // 5
    train, val, test = events[:-2 * n_test_int], events[-2 * n_test_int:-n_test_int], events[-n_test_int:]
    # events = preprocess(train)
    scores = actr_score(bll, assoc, train)
    events = scores.reset_index()
    events['user'] = [test_user for _ in range(len(events))]
    events['track_id'] = events['index']
    events['value'] = events[0]
    events = events[['track_id', 'value', 'user']]
    events = events.append(df_dummies)
    events = events.groupby(['user', 'track_id'])['value'].aggregate('sum').unstack(fill_value=0)
    events = events.to_numpy()
    events = events.reshape(1, -1)
    events = coo_matrix(events)
    events_ = vstack([events_, events])
    events_ = coo_matrix(events_)

In [ ]:
events_ = csr_matrix(events_)
#sparse.save_npz(datadir+"user_item_matrix_actr.npz", events_)

# Generate recency weighted user-item matrix from train data (not used in the paper)
 (can be skipped if the files are already available)

In [ ]:
recency_para = 0

listi = []  #this collects all items of any user and inserts 0 and will be appended to each users history to get the full user-item matrix

i = 0
for test_user in users:
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    col_names = ['user', 'track_id', 'album_id', 'timestamp']
    events = pd.read_csv(filename, names=col_names,
                         quoting=3, sep="\t", header=None, encoding='utf-8'
                         )
    events = events[events["track_id"].isin(items_above_threshold)]
    listi = listi + (events['track_id'].unique().tolist())
    i += 1

listi = list(dict.fromkeys(listi))

df_dummies = pd.DataFrame(listi, columns=['track_id'])
df_dummies['value'] = [0 for _ in range(len(df_dummies))]

#start with 1 user and create dictionary, listening events will get value 1, non listening 0 via dummies, 
#events_ will be the user rating matrix
for test_user in [2]:
    df_dummies['user'] = [test_user for _ in range(len(df_dummies))]
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    events = load_file(filename)
    events = events[events["track_id"].isin(items_above_threshold)]
    user_interactions = len(events)
    n_test_int = user_interactions // 5
    train, val, test = events[:-2 * n_test_int], events[-2 * n_test_int:-n_test_int], events[-n_test_int:]
    events = train

    events['value'] = [1 for _ in range(len(events))]
    i = 0
    length = len(events)
    for index, row in events.iterrows():
        events.at[index, 'value'] = (length - i) ** (-recency_para)
        i += 1
    events = events[['track_id', 'value', 'user']]
    events = events.append(df_dummies)
    events_ = events.groupby(['user', 'track_id'])['value'].aggregate('sum').unstack(fill_value=0)
    dictionary_track_id = pd.DataFrame(events_.columns.values.tolist(), columns=['track_id'])
    #dictionary_track_id.to_csv('dictionary_track_id_recency_500.csv')

    events_ = events_.to_numpy()
    events_ = events_.reshape(1, -1)
    events_ = coo_matrix(events_)
users.remove(2)
for test_user in users:
    df_dummies['user'] = [test_user for _ in range(len(df_dummies))]
    filename = datadir + f"user_split/listening_events_2019_{test_user}.tsv"
    events = load_file(filename)
    events = events[events["track_id"].isin(items_above_threshold)]
    user_interactions = len(events)
    n_test_int = user_interactions // 5
    train, val, test = events[:-2 * n_test_int], events[-2 * n_test_int:-n_test_int], events[-n_test_int:]
    events = train
    events['value'] = [1 for _ in range(len(events))]
    i = 0
    length = len(events)
    for index, row in events.iterrows():
        events.at[index, 'value'] = (length - i) ** (-recency_para)
        i += 1
    events = events[['track_id', 'value', 'user']]
    events = events.append(df_dummies)
    events = events.groupby(['user', 'track_id'])['value'].aggregate('sum').unstack(fill_value=0)
    events = events.to_numpy()
    events = events.reshape(1, -1)
    events = coo_matrix(events)
    events_ = vstack([events_, events])
    events_ = coo_matrix(events_)